In [4]:
!pip install nass

  Created wheel for nass: filename=nass-0.1.1-py2.py3-none-any.whl size=6849 sha256=220bb698d64833a316fcbd9369682e49d7317f8333f2411214e8f046e668195b
  Stored in directory: c:\users\shuoy\appdata\local\pip\cache\wheels\c3\e7\33\6e640077273f8d5094900f1b7df92249b1ffc749d1e8c3b2c9
Successfully built nass


In [5]:
import pandas as pd
import nass
import seaborn as sns
import matplotlib.pyplot as plt

### 1. USDA NASS API Login
Please get an API key here:
https://quickstats.nass.usda.gov/api

In [6]:
api = nass.NassApi('FADC6D51-4CEC-3CE7-89A6-BB9DB2C4B158')
q = api.query()

### 2. Download yield data for all counties
Since we can only retrieve 5000 entries for each api query, I tried to get all data using a for loop.
May check category names at https://quickstats.nass.usda.gov/.

In [ ]:
# Import cencus data for county ID.
filename = 'Data_original\\All_counties.xlsx'
All_counties_df = pd.read_excel(filename, engine='openpyxl', header=0)

In [8]:
All_counties_df.Areaname = All_counties_df.Areaname.astype(str)

In [9]:
All_counties_df['county_name'] = [x[:-4].upper() for x in All_counties_df['Areaname']]
All_counties_df['state_abr'] = [x[-2:].upper() for x in All_counties_df['Areaname']]
All_counties_df['state_code'] = ['{0:02}'.format(x//1000) for x in All_counties_df['STCOU']]
All_counties_df['county_code'] = ['{0:03}'.format(x - 1000*(x//1000)) for x in All_counties_df['STCOU']]

In [10]:
All_counties_df = All_counties_df[All_counties_df['county_code']!='000']

In [11]:
All_counties_df.county_code = All_counties_df.county_code.astype(str)
All_counties_df.state_code = All_counties_df.state_code.astype(str)

In [12]:
All_counties_df.reset_index(drop=True, inplace=True)

In [13]:
All_counties_df

,Areaname,STCOU,county_name,state_abr,state_code,county_code
0,"Autauga, AL",1001,AUTAUGA,AL,01,001
1,"Baldwin, AL",1003,BALDWIN,AL,01,003
2,"Barbour, AL",1005,BARBOUR,AL,01,005
3,"Bibb, AL",1007,BIBB,AL,01,007
4,"Blount, AL",1009,BLOUNT,AL,01,009
...,...,...,...,...,...,...
3141,"Sweetwater, WY",56037,SWEETWATER,WY,56,037
3142,"Teton, WY",56039,TETON,WY,56,039
3143,"Uinta, WY",56041,UINTA,WY,56,041
3144,"Washakie, WY",56043,WASHAKIE,WY,56,043


In [14]:
def get_data_USDA(category):
    count = 0
    for i in range(len(All_counties_df)):
        state_code = All_counties_df.loc[i, 'state_code']
        county_code = All_counties_df.loc[i, 'county_code']
        q = api.query()
        q.filter('state_fips_code', state_code).filter('county_code', county_code).filter('short_desc', category)
        try:
            df = pd.DataFrame(q.execute())
            if count == 0:
                output_df = df
            else:
                output_df = pd.concat([output_df, df], ignore_index=True)
            count += 1
        except Exception:
            print(All_counties_df.loc[i, 'Areaname'])
            pass
    
    return output_df

In [ ]:
corn_yield_data = get_data_USDA('CORN, GRAIN - YIELD, MEASURED IN BU / ACRE')

In [ ]:
corn_yield_data.to_csv('corn_yield_data.csv')

In [ ]:
soybeans_yield_data = get_data_USDA('SOYBEANS - YIELD, MEASURED IN BU / ACRE')

In [ ]:
soybeans_yield_data.to_csv('soybeans_yield_data.csv')

In [ ]:
sorghum_yield_data = get_data_USDA('SORGHUM, GRAIN - YIELD, MEASURED IN BU / ACRE')

In [ ]:
sorghum_yield_data.to_csv('sorghum_yield_data.csv')

### 3. Download ag rent data for each county
Category names are "RENT, CASH, CROPLAND, IRRIGATED - EXPENSE, MEASURED IN $\$$ / ACRE", "RENT, CASH, CROPLAND, NON-IRRIGATED - EXPENSE, MEASURED IN $\$$ / ACRE", and "RENT, CASH, PASTURELAND - EXPENSE, MEASURED IN $\$$ / ACRE".

In [ ]:
nonirrigated_rent_data = get_data_USDA('RENT, CASH, CROPLAND, NON-IRRIGATED - EXPENSE, MEASURED IN $ / ACRE')

Aleutians East, AK
Aleutians West, AK
Anchorage, AK
Bethel, AK
Bristol Bay, AK
Denali, AK
Dillingham, AK
Fairbanks North Star, AK
Haines, AK
Hoonah-Angoon, AK
Juneau, AK
Kenai Peninsula, AK
Ketchikan Gateway, AK
Kodiak Island, AK
Lake and Peninsula, AK
Matanuska-Susitna, AK
Nome, AK
North Slope, AK
Northwest Arctic, AK
Petersburg, AK
Prince of Wales-Hyder, AK
Sitka, AK
Skagway, AK
Southeast Fairbanks, AK
Valdez-Cordova, AK
Wade Hampton, AK
Wrangell, AK
Yakutat, AK
Yukon-Koyukuk, AK
Apache, AZ
Cochise, AZ
Coconino, AZ
Gila, AZ
Graham, AZ
Greenlee, AZ
La Paz, AZ
Maricopa, AZ
Mohave, AZ
Navajo, AZ
Pima, AZ
Pinal, AZ
Santa Cruz, AZ
Yavapai, AZ
Yuma, AZ
Dallas, AR
St. Francis, AR
Alpine, CA
Del Norte, CA
Imperial, CA
Lake, CA
Los Angeles, CA
Mono, CA
Orange, CA
Plumas, CA
San Bernardino, CA
San Diego, CA
San Francisco, CA
Sierra, CA
Trinity, CA
Clear Creek, CO
Crowley, CO
Custer, CO
Denver, CO
Dolores, CO
Eagle, CO
Gilpin, CO
Gunnison, CO
Hinsdale, CO
Huerfano, CO
Lake, CO
Mineral, CO
Ouray